## Loading the grid data 

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('/Users/hassaanulhaq/Library/Mobile Documents/com~apple~CloudDocs/spring_2025/GIS/final_project/clone_gis/gis_spatiotemporal_hybrid/grid_data/grid_monthly_data.csv')

In [3]:
df.columns

Index(['Month_Year', 'grid_id', '2.5', 'CO', 'NO', 'PM10', 'SO2', 'month',
       'year'],
      dtype='object')

In [5]:
print(df.head())

   Month_Year grid_id       2.5        CO        NO      PM10       SO2  \
0  2011-01-01    10_2  1.321495 -0.335843  0.269377 -0.779806  0.547902   
1  2011-01-01    1_-2       NaN       NaN       NaN       NaN  2.618263   
2  2011-01-01     1_4  1.687086       NaN       NaN       NaN       NaN   
3  2011-01-01     1_9  1.297026       NaN       NaN -0.665755       NaN   
4  2011-01-01     3_4  1.455528       NaN  0.929956       NaN  0.945232   

      month     year  
0 -1.599228 -1.69753  
1 -1.599228 -1.69753  
2 -1.599228 -1.69753  
3 -1.599228 -1.69753  
4 -1.599228 -1.69753  


In [7]:
import os

monthly_folder = '/Users/hassaanulhaq/Library/Mobile Documents/com~apple~CloudDocs/spring_2025/GIS/final_project/clone_gis/gis_spatiotemporal_hybrid/monthly'
csv_files = [f for f in os.listdir(monthly_folder) if f.endswith('.csv')]

for file in csv_files:
    file_path = os.path.join(monthly_folder, file)
    temp_df = pd.read_csv(file_path, nrows=1)
    print(f"{file}: {list(temp_df.columns)}")

merged_2.5_monthly.csv: ['Local Site Name', 'Month_Year', 'monthly_value', 'Site Latitude', 'Site Longitude', 'pollutant']
merged_co_monthly.csv: ['Local Site Name', 'Month_Year', 'monthly_value', 'Site Latitude', 'Site Longitude', 'pollutant']
merged_so2_monthly.csv: ['Local Site Name', 'Month_Year', 'monthly_value', 'Site Latitude', 'Site Longitude', 'pollutant']
merged_no_monthly.csv: ['Local Site Name', 'Month_Year', 'monthly_value', 'Site Latitude', 'Site Longitude', 'pollutant']
merged_pm10_monthly.csv: ['Local Site Name', 'Month_Year', 'monthly_value', 'Site Latitude', 'Site Longitude', 'pollutant']


In [9]:
files = {
    'PM2.5': 'monthly/merged_2.5_monthly.csv',
    'CO': 'monthly/merged_co_monthly.csv',
    'SO2': 'monthly/merged_so2_monthly.csv',
    'NO': 'monthly/merged_no_monthly.csv',
    'PM10': 'monthly/merged_pm10_monthly.csv'
}
dfs = [pd.read_csv(f).assign(pollutant=name) for name, f in files.items()]
combined_df = pd.concat(dfs).pivot_table(index=['Local Site Name', 'Month_Year', 'Site Latitude', 'Site Longitude'],
                                        columns='pollutant', values='monthly_value')


In [11]:
print(combined_df.head())

pollutant                                                   CO  NO  PM10  \
Local Site Name    Month_Year Site Latitude Site Longitude                 
4TH DISTRICT COURT 2011-01    41.872897     -87.825872     NaN NaN   NaN   
                   2011-02    41.872897     -87.825872     NaN NaN   NaN   
                   2011-03    41.872897     -87.825872     NaN NaN   NaN   
                   2011-04    41.872897     -87.825872     NaN NaN   NaN   
                   2011-05    41.872897     -87.825872     NaN NaN   NaN   

pollutant                                                       PM2.5  SO2  
Local Site Name    Month_Year Site Latitude Site Longitude                  
4TH DISTRICT COURT 2011-01    41.872897     -87.825872      57.870968  NaN  
                   2011-02    41.872897     -87.825872      52.440000  NaN  
                   2011-03    41.872897     -87.825872      49.064516  NaN  
                   2011-04    41.872897     -87.825872      51.448276  NaN  
     

In [12]:
combined_df.to_csv('/Users/hassaanulhaq/Library/Mobile Documents/com~apple~CloudDocs/spring_2025/GIS/final_project/clone_gis/gis_spatiotemporal_hybrid/grid_data/combined.csv', index=True)

### In this we are basically using combined_df because now each pollutant is a column, and we have data for each pollutant. Now we convert it to grid

In [15]:
import numpy as np

In [20]:
from pykrige.ok import OrdinaryKriging

df_reset = combined_df.reset_index()

df_clean = df_reset.dropna(subset=['PM2.5', 'Site Latitude', 'Site Longitude'])

print(f"Number of valid data points: {len(df_clean)}")

if len(df_clean) > 0:
    grid_lat = np.arange(df_clean['Site Latitude'].min(), df_clean['Site Latitude'].max(), 0.1)
    grid_lon = np.arange(df_clean['Site Longitude'].min(), df_clean['Site Longitude'].max(), 0.1)
    
    OK = OrdinaryKriging(df_clean['Site Longitude'], df_clean['Site Latitude'], df_clean['PM2.5'])
    z, _ = OK.execute('grid', grid_lon, grid_lat)
    print("Kriging interpolation completed successfully")
else:
    print("No valid data points found for interpolation")


Number of valid data points: 1678
Kriging interpolation completed successfully


In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, Reshape, Concatenate
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Prepare latlon_coords from df_clean
latlon_coords = df_clean[['Site Latitude', 'Site Longitude']].to_numpy()

# Define calculate_knn using sklearn
def calculate_knn(coords, k=5):
    nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='ball_tree').fit(coords)
    distances, indices = nbrs.kneighbors(coords)
    # Exclude self (first neighbor is the point itself)
    return indices[:, 1:]

# Spatial KNN component (k=5 nearest neighbors)
knn_indices = calculate_knn(latlon_coords, k=5)

# Define model input dimensions (example values, adjust as needed)
seq_length = 12  # e.g., 12 months
grid_h = 50
grid_w = 50
num_features = 5 # e.g., only PM2.5, or set to number of pollutants

# ConvLSTM backbone
from tensorflow.keras.layers import Dense

model = Sequential([
    ConvLSTM2D(filters=64, kernel_size=(3,3), 
               input_shape=(seq_length, grid_h, grid_w, num_features),
               padding='same', return_sequences=True),

    # SpatioTemporalAttention(),  # Custom attention layer (removed or define if needed)
    ConvLSTM2D(filters=64, kernel_size=(3,3), return_sequences=False, padding='same'),
    Dense(1)  # Replace with prediction_horizon if defined
])


/Users/hassaanulhaq/Library/Mobile Documents/com~apple~CloudDocs/spring_2025/GIS/final_project/clone_gis/gis_spatiotemporal_hybrid/myenv12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [26]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d_2 (ConvLSTM2D)      │ (None, 12, 50, 50, 64) │       159,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_3 (ConvLSTM2D)      │ (None, 50, 50, 64)     │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50, 50, 1)      │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 454,465 (1.73 MB)

 Trainable params: 454,465 (1.73 MB)

 Non-trainable params: 0 (0.00 B)

### Above is a functional convlstm. We need to add KNN and STA to it

In [42]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
from sklearn.neighbors import NearestNeighbors

# ======================
# 1. Enhanced KSC Components - FIXED VERSION
# ======================

class KNNReducer(layers.Layer):
    """Properly handles batch and time dimensions for spatial filtering"""
    def __init__(self, knn_indices_1d, **kwargs):
        super().__init__(**kwargs)
        # Store 1D indices directly
        self.knn_indices = tf.constant(knn_indices_1d, dtype=tf.int32)  # Shape (h*w, k)
        
    def call(self, inputs):
        # Input shape: (batch, time, h, w, features)
        batch, time, h, w, features = tf.unstack(tf.shape(inputs))
        
        # Reshape to treat batch-time as single dimension
        x = tf.reshape(inputs, [batch*time, h, w, features])
        
        # Gather neighbors for each spatial position
        flat_x = tf.reshape(x, [batch*time, h*w, features])
        neighbors = tf.gather(flat_x, self.knn_indices, axis=1)  # (batch*time, h*w, k, features)
        
        # Average across neighbors
        reduced = tf.reduce_mean(neighbors, axis=2)
        
        # Reshape back to original dimensions
        return tf.reshape(reduced, [batch, time, h, w, features])

class SpatioTemporalAttention(layers.Layer):
    def build(self, input_shape):
        self.time_attention = layers.Attention(use_scale=True)
        self.spatial_attention = layers.Attention(use_scale=True)

    def call(self, inputs):
        batch, time, h, w, features = tf.unstack(tf.shape(inputs))
        x = tf.reshape(inputs, [batch, time, h * w, features])
        x = tf.transpose(x, [0, 2, 1, 3])
        x_reshaped = tf.reshape(x, [-1, time, features])
        t_att = self.time_attention([x_reshaped, x_reshaped])
        t_att = tf.reshape(t_att, [batch, h * w, time, features])
        t_att = tf.transpose(t_att, [0, 2, 1, 3])
        t_att = tf.reshape(t_att, [batch, time, h, w, features])

        x = tf.reshape(t_att, [batch, time, h * w, features])
        x_reshaped = tf.reshape(x, [-1, h * w, features])
        s_att = self.spatial_attention([x_reshaped, x_reshaped])
        return tf.reshape(s_att, [batch, time, h, w, features])

def residual_block(x):
    shortcut = x
    x = layers.LayerNormalization()(x)
    x = layers.ConvLSTM2D(64, (3,3), padding='same', return_sequences=True)(x)
    return layers.Add()([shortcut, x])

# ======================
# 2. Dimension-Correct Model
# ======================

def build_ksc_convlstm(knn_indices_1d, grid_h, grid_w, seq_length=12, features=5):
    inputs = layers.Input(shape=(seq_length, grid_h, grid_w, features))
    
    # 1. KNN Spatial Filtering
    x = KNNReducer(knn_indices_1d)(inputs)
    
    # 2. ConvLSTM backbone
    x = layers.ConvLSTM2D(
        64, (3,3), padding='same', 
        return_sequences=True, 
        activation='tanh'
    )(x)
    
    # 3. Residual blocks with attention
    for _ in range(2):
        x = residual_block(x)
    x = SpatioTemporalAttention()(x)
    
    # 4. Final prediction
    x = layers.ConvLSTM2D(64, (3,3), padding='same', return_sequences=False)(x)
    outputs = layers.Dense(1, activation='relu')(x)
    
    return Model(inputs, outputs)

# ======================
# 3. Usage with Air Quality Data
# ======================

def create_knn_indices(grid_h, grid_w, k=5):
    """Generates 1D spatial indices for KNN filtering"""
    x = np.arange(grid_h)
    y = np.arange(grid_w)
    xx, yy = np.meshgrid(x, y, indexing='ij')
    coords = np.column_stack([xx.ravel(), yy.ravel()])
    
    nbrs = NearestNeighbors(n_neighbors=k+1).fit(coords)
    _, indices = nbrs.kneighbors(coords)
    return indices[:, 1:]  # Exclude self (first neighbor)

# Configuration
GRID_H, GRID_W = 50, 50
SEQ_LENGTH = 12
FEATURES = 5
K = 5

# 1. Generate KNN indices
knn_1d = create_knn_indices(GRID_H, GRID_W, k=K)

# 2. Initialize model
model = build_ksc_convlstm(
    knn_indices_1d=knn_1d,
    grid_h=GRID_H,
    grid_w=GRID_W,
    seq_length=SEQ_LENGTH,
    features=FEATURES
)

# 3. Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.Huber(delta=1.0),
    metrics=['mae', 'mse']
)

model.summary()

# Test with dummy data
dummy_input = tf.random.normal((32, 12, 50, 50, 5))
dummy_output = model(dummy_input)
print(dummy_output.shape)  # Should output (32, 50, 50, 1)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 12, 50,    │          0 │ -                 │
│ (InputLayer)        │ 50, 5)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ knn_reducer_3       │ (None, 12, 50,    │          0 │ input_layer_6[0]… │
│ (KNNReducer)        │ 50, 5)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d_16      │ (None, 12, 50,    │    159,232 │ knn_reducer_3[0]… │
│ (ConvLSTM2D)        │ 50, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 12, 50,    │        128 │ conv_lstm2d_16[0… │
│ (LayerNormalizatio… │ 50, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d_17      │ (None, 12, 50,    │    295,168 │ layer_normalizat… │
│ (ConvLSTM2D)        │ 50, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_6 (Add)         │ (None, 12, 50,    │          0 │ conv_lstm2d_16[0… │
│                     │ 50, 64)           │            │ conv_lstm2d_17[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 12, 50,    │        128 │ add_6[0][0]       │
│ (LayerNormalizatio… │ 50, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d_18      │ (None, 12, 50,    │    295,168 │ layer_normalizat… │
│ (ConvLSTM2D)        │ 50, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_7 (Add)         │ (None, 12, 50,    │          0 │ add_6[0][0],      │
│                     │ 50, 64)           │            │ conv_lstm2d_18[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatio_temporal_at… │ (None, 12, 50,    │          2 │ add_7[0][0]       │
│ (SpatioTemporalAtt… │ 50, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d_19      │ (None, 50, 50,    │    295,168 │ spatio_temporal_… │
│ (ConvLSTM2D)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 50, 50, 1) │         65 │ conv_lstm2d_19[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,045,059 (3.99 MB)

 Trainable params: 1,045,059 (3.99 MB)

 Non-trainable params: 0 (0.00 B)

(32, 50, 50, 1)


In [43]:
dummy_input = tf.random.normal((32, 12, 50, 50, 5))
dummy_output = model(dummy_input)
print(dummy_output.shape)  # Now outputs (32, 50, 50, 1)


(32, 50, 50, 1)


In [ ]:
# Assuming your grid data is in variable z (shape: (lat, lon)), and you want to use the trained model to predict on this grid.
# The model expects input shape: (batch, seq_length, grid_h, grid_w, num_features)
# We'll create a dummy sequence for demonstration, using z as the last time step and filling previous steps with zeros.

# 1. Prepare grid data for model input
# If z is masked, fill masked values with 0 or np.nan as appropriate
z_filled = np.ma.filled(z, fill_value=0)

# 2. Create a sequence (e.g., repeat z for seq_length steps, or use zeros for previous steps)
# Here, we create a batch of 1, seq_length=12, grid_h, grid_w, num_features=5
# We'll fill only the last feature (e.g., PM2.5) with z, others with zeros
input_grid = np.zeros((1, seq_length, grid_h, grid_w, num_features), dtype=np.float32)
input_grid[0, -1, :z_filled.shape[0], :z_filled.shape[1], 0] = z_filled  # Fill PM2.5 at last time step

# 3. Predict using the model
grid_pred = model.predict(input_grid)
print("Prediction shape:", grid_pred.shape)